In [1]:
import os
from os.path import join

import numpy as np
from numpy import flatnonzero as find
from scipy.sparse import hstack, vstack
from scipy.sparse.linalg import spsolve

import pypower
from pypower.api import case9, loadcase, ext2int, makeYbus, makeSbus, bustypes
from pypower.idx_bus import PD, QD, VM, VA, GS, BUS_TYPE, PQ, REF
from pypower.idx_brch import PF, PT, QF, QT
from pypower.idx_gen import PG, QG, VG, QMAX, QMIN, GEN_BUS, GEN_STATUS
from pypower.dSbus_dV import dSbus_dV

In [2]:
# list of available cases
['case4gs', 'case6ww', 'case9',
'case9Q', 'case14', 'case24_ieee_rts',
'case30', 'case30Q', 'case30pwl',
'case39', 'case57', 'case118', 'case300',
'case30_userfcns']

casename = 'case57'
casedata = join('pypower', casename)
data = ext2int(loadcase(casedata))

In [3]:
baseMVA = data['baseMVA']
bus = data['bus']
branch = data['branch']
gen = data['gen']

ref, pv, pq = bustypes(bus, gen)
pvpq = np.r_[pv, pq]
npv = len(pv)
npq = len(pq)
j1 = 0;         j2 = npv           ## j1:j2 - V angle of pv buses
j3 = j2;        j4 = j2 + npq      ## j3:j4 - V angle of pq buses
j5 = j4;        j6 = j4 + npq      ## j5:j6 - V mag of pq buses

In [4]:
def get_V0(bus, gen):
    on = find(gen[:, GEN_STATUS] > 0)      ## which generators are on?
    gbus = gen[on, GEN_BUS].astype(int) 
    V0  = bus[:, VM] * np.exp(1j * np.pi/180 * bus[:, VA])
    V0[gbus] = gen[on, VG] / abs(V0[gbus]) * V0[gbus]
    return V0

def mismatch(Y, V, S):
    return V * np.conj(Y * V) - S

def mismatchf(Y, V, S):
    mis = mismatch(Y, V, S)
    F = np.r_[  mis[pv].real,
             mis[pq].real,
             mis[pq].imag  ]
    return F

def jacobian(Y, V):
    dS_dVm, dS_dVa = dSbus_dV(Y, V)
    J11 = dS_dVa[pvpq[:, None], pvpq].real
    J12 = dS_dVm[pvpq[:, None], pq].real
    J21 = dS_dVa[pq[:, None], pvpq].imag
    J22 = dS_dVm[pq[:, None], pq].imag

    J = vstack([
            hstack([J11, J12]),
            hstack([J21, J22])
        ], format="csr")
    
    return J

def updatedV(V, dx):
    Vm = np.abs(V)
    Va = np.angle(V)
    if npv:
        Va[pv] = Va[pv] + dx[j1:j2]
    if npq:
        Va[pq] = Va[pq] + dx[j3:j4]
        Vm[pq] = Vm[pq] + dx[j5:j6]
    V = Vm * np.exp(1j * Va)
    return V

def norm(F):
    return np.linalg.norm(F, np.Inf)

$j$ denotes imaginary unit in electrical engineering  
$Y_{bus} = Y = G + jB$  
$S_{bus} = S = P + jQ$  
$V_i = |V_i|e^{j\delta_i}$  
$\theta_{ik} = \delta_i - \delta_k$

$S_k = V_k \cdot (YV)_k^*$  
or   
$P_{i}=\sum _{{k=1}}^{N}|V_{i}||V_{k}|(G_{{ik}}\cos \theta _{{ik}}+B_{{ik}}\sin \theta _{{ik}})$    
$Q_{i}=\sum _{{k=1}}^{N}|V_{i}||V_{k}|(G_{{ik}}\sin \theta _{{ik}}-B_{{ik}}\cos \theta _{{ik}})$  
Further reading: https://wiki.openelectrical.org/index.php?title=Power_Flow  
### Newton algorthm
$$
\Delta P_{i}=-P_{i}+\sum _{{k=1}}^{N}|V_{i}||V_{k}|(G_{{ik}}\cos \theta _{{ik}}+B_{{ik}}\sin \theta _{{ik}})
\\
\Delta Q_{{i}}=-Q_{{i}}+\sum _{{k=1}}^{N}|V_{i}||V_{k}|(G_{{ik}}\sin \theta _{{ik}}-B_{{ik}}\cos \theta _{{ik}})
\\
J={\begin{bmatrix}{\dfrac  {\partial \Delta P}{\partial \theta }}&{\dfrac  {\partial \Delta P}{\partial |V|}}\\{\dfrac  {\partial \Delta Q}{\partial \theta }}&{\dfrac  {\partial \Delta Q}{\partial |V|}}\end{bmatrix}}
\\
{\begin{bmatrix}\Delta \theta \\\Delta |V|\end{bmatrix}}=-J^{{-1}}{\begin{bmatrix}\Delta P\\\Delta Q\end{bmatrix}}
\\
\theta ^{{m+1}}=\theta ^{m}+\Delta \theta \,
\\
|V|^{{m+1}}=|V|^{m}+\Delta |V|\,$$   
**What is not clarified yet**: types of buses: $pv$ and $pq$

In [5]:
Ybus, _, _ = makeYbus(baseMVA, bus, branch)
Sbus = makeSbus(baseMVA, bus, gen)
V0 = get_V0(bus, gen)

In [14]:
V = V0
F = mismatchf(Ybus, V, Sbus)

In [20]:
F.shape

(106,)

In [21]:
J.shape 

(106, 106)

In [6]:
V = V0
F = mismatchf(Ybus, V, Sbus)
normF = norm(F)
print(normF)

for _ in range(5):
    J = jacobian(Ybus, V)
    F = mismatchf(Ybus, V, Sbus)
    dx = -spsolve(J, F)
    V = updatedV(V, dx)
    normF = norm(F)
    print(normF)

0.45788509719535103
0.45788509719535103
0.005380113910526581
5.221329097899152e-06
3.449698351981805e-12
1.3189296159535811e-14


In [20]:
V

array([1.04      +0.j        , 1.00978284-0.02094323j,
       0.97962538-0.10275753j, 0.97274838-0.12525678j,
       0.96565546-0.14511792j, 0.96879087-0.14779802j,
       0.9755528 -0.13019076j, 1.00193224-0.07846514j,
       0.96631981-0.16317486j, 0.96661532-0.19577581j,
       0.95858973-0.17236091j, 0.99809655-0.18446758j,
       0.96459316-0.16667517j, 0.95728654-0.15762484j,
       0.98026192-0.12366501j, 1.00127964-0.15606106j,
       1.01294563-0.0956783j , 0.97976324-0.20342784j,
       0.94442262-0.22197332j, 0.93737882-0.22408132j,
       0.98293086-0.22564498j, 0.98436023-0.22498411j,
       0.9827252 -0.2257883j , 0.97246409-0.22973956j,
       0.93351058-0.30643936j, 0.93431431-0.21538156j,
       0.96178983-0.19591646j, 0.98004677-0.18131556j,
       0.99556321-0.17145866j, 0.91173684-0.30895437j,
       0.88288053-0.31063084j, 0.90072375-0.30159361j,
       0.89833986-0.30148729j, 0.93010124-0.23447046j,
       0.93789276-0.23221256j, 0.94832725-0.23003458j,
       0.9